In [1]:
"""
#6.9 Map Reduce LCEL Chain (19:48)

- Retriever 가 검색 결과로 천 개 이상의 document를 반환한다면, stuff는 사용할 수 없음.
-

"""

'\n#6.7 Recap (05:17)\n#6.8 Stuff LCEL Chain (10:13)\n\n1. File들을 Load 하기.\n - TextLoader, UnstructuredFileLoader\n2. Split (거대한 단일 document 보다 작은 여러개를 LLM 전달할 때 검색 성능이 좋다\n3. Ebmedding : text에 의미별로 적절한 점수를 부여해서 vector 형식으로 표현한거\n - O\n\n'

In [13]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import  UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    temperature=0.1,
)
cache_dir = LocalFileStore("./.cache/")

# chunk_size : 얼마나 큰 덩어리로 나눌지 결정할 수 있음
# chunk_overlap : 문장이나 문단을 분할할 때 앞 조각 일부분을 가져옴
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/Romeo and Juliet by William Shakespeare.txt",encoding="utf-8")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)


In [23]:
retriever  = vectorstore.as_retriever()


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer.
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("What was the reason they fell in love?")

AIMessage(content="They fell in love because Romeo's heart was set on the fair daughter of rich Capulet, and her heart was set on his.")